In [47]:
# read a mesh from file capacitor.msh
from mpi4py import MPI
from dolfinx.io import gmshio
domain, cell_tags, facet_tags = gmshio.read_from_msh("circle-capacitor.msh", MPI.COMM_WORLD, 0, gdim=2)

Info    : Reading 'circle-capacitor.msh'...
Info    : 21 entities
Info    : 1325 nodes
Info    : 2661 elements
Info    : Done reading 'circle-capacitor.msh'


In [58]:
# define finite element function space
from dolfinx.fem import functionspace
V = functionspace(domain, ("Lagrange", 1))

# identify the boundary (create facet to cell connectivity required to determine boundary facets)
from dolfinx import default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)

# Find facets marked with 2 and 3 (the two rectangles)
facets_rect1 = facet_tags.find(11)
facets_rect2 = facet_tags.find(12)

# Locate degrees of freedom
dofs_rect1 = locate_dofs_topological(V, fdim, facets_rect1)
dofs_rect2 = locate_dofs_topological(V, fdim, facets_rect2)

# Define different Dirichlet values
u_rect1 = Constant(domain, 0.0)
u_rect2 = Constant(domain, 2.0)

# Create BCs
bc1 = dirichletbc(u_rect1, dofs_rect1, V)
bc2 = dirichletbc(u_rect2, dofs_rect2, V)

bcs = [bc1, bc2]

# trial and test functions
import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# source term
from dolfinx import default_scalar_type
from dolfinx import fem
f = fem.Constant(domain, default_scalar_type(-3.0))

# variational problem
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

# assemble the system
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [ ]:
# Save force (derivative of solution) to csv file
import csv
import os
import gmsh
import numpy as np

filename = "force.csv"
folder = "results"

#create a CSV file
os.makedirs(folder, exist_ok=True)
filepath = os.path.join(folder, filename)

# Define the column headers
headers = ['x', 'y', 'z', 'Fx', 'Fy', 'Fz']
# Write the headers to the CSV file
with open(filepath, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)

# Ensure Gmsh is initialized
if not gmsh.isInitialized():
    gmsh.initialize()

segment_tag = 15 # tag of the segment where we want to save the force

facet_indices = np.where(facet_tags.values == segment_tag)[0]
facets = facet_tags.indices[facet_indices]
domain.topology.create_connectivity(domain.topology.dim - 1, 0)
f_to_v = domain.topology.connectivity(domain.topology.dim - 1, 0)
coords = domain.geometry.x
segment_coords = []

for facet in facets:
    vertex_indices = f_to_v.links(facet)
    vertex_coords = coords[vertex_indices]
    segment_coords.append(vertex_coords)

seen = set()  # This will store unique points as tuples
flattened_coords = []  # This will store the unique points in flattened form

for coord_set in segment_coords:
    for point in coord_set:
        point = np.array(point)
        flattened_coords.append(point)  # Add to the flattened list
segment_coords = np.array(flattened_coords, dtype=np.float64)

#remove duplicates

from dolfinx.fem import Function
from ufl.finiteelement import VectorElement
from dolfinx.fem import functionspace
element = VectorElement("Lagrange", domain.ufl_cell(), 1)
V_grad = functionspace(domain, element)
grad_uh_expr = fem.Expression(ufl.grad(uh), V_grad.element.interpolation_points())
grad_uh = fem.Function(V_grad)
grad_uh.interpolate(grad_uh_expr)

from dolfinx.geometry import BoundingBoxTree, compute_collisions_points, compute_colliding_cells

# Build a bounding box tree for fast point location
tree = BoundingBoxTree(domain, domain.topology.dim)

# Find cells containing the points
collisions = compute_collisions_points(tree, segment_coords)
cells = compute_colliding_cells(domain, collisions)

# Evaluate gradient in each cell (will be nan if point is outside the domain)
values = np.zeros((segment_coords.shape[0], domain.geometry.dim), dtype=np.float64)
for i, (pt, cell) in enumerate(zip(segment_coords, cells)):
    if cell >= 0:
        values[i] = grad_uh.eval(pt, cell)
    else:
        values[i] = np.nan  # point is outside the domain


from force_getter import save_force_on_segment
save_force_on_segment(segment_coords, values, filename, folder)

KeyboardInterrupt: 

In [55]:
print(segment_coords)

[[-1.6375 -0.5     0.    ]
 [-1.575  -0.5     0.    ]
 [-1.6375 -0.5     0.    ]
 [-1.7    -0.5     0.    ]
 [-1.575  -0.5     0.    ]
 [-1.5125 -0.5     0.    ]
 [-1.7    -0.5     0.    ]
 [-1.7625 -0.5     0.    ]
 [-1.5125 -0.5     0.    ]
 [-1.45   -0.5     0.    ]
 [-1.45   -0.5     0.    ]
 [-1.3875 -0.5     0.    ]
 [-1.7625 -0.5     0.    ]
 [-1.825  -0.5     0.    ]
 [-1.825  -0.5     0.    ]
 [-1.8875 -0.5     0.    ]
 [-1.3875 -0.5     0.    ]
 [-1.325  -0.5     0.    ]
 [-1.325  -0.5     0.    ]
 [-1.2625 -0.5     0.    ]
 [-1.8875 -0.5     0.    ]
 [-1.95   -0.5     0.    ]
 [-1.95   -0.5     0.    ]
 [-2.0125 -0.5     0.    ]
 [-1.2625 -0.5     0.    ]
 [-1.2    -0.5     0.    ]
 [-2.0125 -0.5     0.    ]
 [-2.075  -0.5     0.    ]
 [-2.075  -0.5     0.    ]
 [-2.1375 -0.5     0.    ]
 [-2.1375 -0.5     0.    ]
 [-2.2    -0.5     0.    ]]


In [25]:
# plot the mesh 
import pyvista
pyvista.OFF_SCREEN = False
print(pyvista.global_theme.jupyter_backend)

from dolfinx import plot
pyvista.start_xvfb()
domain.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=False)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")


trame


Widget(value='<iframe src="http://localhost:34645/index.html?ui=P_0x7f6a90e9bc40_9&reconnect=auto" class="pyvi…

In [26]:
# plot the solution
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=False, cmap="bwr")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

Widget(value='<iframe src="http://localhost:34645/index.html?ui=P_0x7f6a90e9a650_10&reconnect=auto" class="pyv…

In [27]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=False, show_scalar_bar=True, cmap="coolwarm")
if not pyvista.OFF_SCREEN:
    plotter2.show()

Widget(value='<iframe src="http://localhost:34645/index.html?ui=P_0x7f6a90e98490_11&reconnect=auto" class="pyv…

In [28]:
from dolfinx import io
from pathlib import Path
results_folder = Path("results")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "fundamentals"
""" with io.VTXWriter(domain.comm, filename.with_suffix(".bp"), [uh]) as vtx:
    vtx.write(0.0) """
with io.VTKFile(domain.comm, filename.with_suffix(".pvd"), "w") as vtk:
    vtk.write_mesh(domain)
    vtk.write_function(uh)
with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(uh)